<a href="https://colab.research.google.com/github/Ismail-Ryad/image-colourization/blob/master/Flask_Trial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from flask import Flask, jsonify, request, send_file, render_template
from PIL import Image
import io
import numpy as np
import torch
import requests
import base64
import torch
import torch.nn as nn

import sys
sys.path.append(".")

#from api.utils import exif_transpose
import PIL.Image
import PIL.ImageOps


class ColorizationUpsampling(nn.Module):
  def __init__(self, input_size=128):
    super(ColorizationUpsampling, self).__init__()
    MIDLEVEL_FEATURE_SIZE = 128

    ## Encoder
    resnet = models.resnet18()
    resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])

    ## Upsampling
    self.upsample = nn.Sequential(     
      nn.Conv2d(MIDLEVEL_FEATURE_SIZE, 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1),
      nn.Upsample(scale_factor=2)
    )

  def forward(self, x):
    ## Encoder
    x = self.midlevel_resnet(x)
    ## Decoder
    x = self.upsample(x)
    return x

from PIL import Image
from matplotlib import pyplot as plt
import torchvision.transforms.functional as TF
import numpy as np
import torch
from skimage.color import lab2rgb, rgb2gray
#from model import ColorizationUpsampling
import cv2
from google.colab.patches import cv2_imshow


def colorizer():
    """
    Predict and show color image from input image
    :return: None
    """
    path = '/content/drive/My Drive/test/gray/img-1-epoch-18.jpg'
    img = Image.open(path)
    img = img.resize((224,224))
    img_original = np.array(img)

    gray = rgb2gray(img_original)
    x = TF.to_tensor(gray).float()
    x.unsqueeze_(0)
    model = ColorizationUpsampling()
    model.load_state_dict(torch.load('/content/drive/My Drive/colorization_models/checkpoints/model-epoch-22-losses-0.002910.pth',
                                     map_location=torch.device('cpu')))

    output = model(x)

    output = output.detach()
    color_image = torch.cat((x[0], output[0]), 0).numpy()
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
    color_image = lab2rgb(color_image.astype(np.float16))

    color_image_bgr = color_image.astype(np.float32)
    color_image_bgr = cv2.cvtColor(color_image_bgr, cv2.COLOR_RGB2BGR)
    color_image_bgr = cv2.resize(color_image_bgr, (380, 240))

    normalized_array = (color_image_bgr - np.min(color_image_bgr)) / (
            np.max(color_image_bgr) - np.min(color_image_bgr))  # this set the range from 0 till 1
    color_image_bgr = (normalized_array * 255).astype(np.uint8)
    gray = cv2.resize(gray, (380, 240))
    gray = np.stack((gray,) * 3, axis=-1)

    gray = (gray - np.min(gray)) / (
            np.max(gray) - np.min(gray))  # this set the range from 0 till 1
    gray = (gray * 255).astype(np.uint8)
    vis = np.concatenate((gray, color_image_bgr), axis=1)

    frame_normed = np.array(vis, np.uint8)

    cv2.imwrite(path[:-4]+"out.jpg", frame_normed)
   # cv2_imshow(frame_normed)
   # cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return frame_normed
def exif_transpose(img: PIL.Image) -> PIL.Image:
    """
    Source: https://github.com/ageitgey/image_to_numpy/blob/master/image_to_numpy/src.py
    """

    exif_orientation_tag = 274

    # Check for EXIF data (only present on some files)
    if hasattr(img, "_getexif") and isinstance(img._getexif(), dict) and exif_orientation_tag in img._getexif():
        exif_data = img._getexif()
        orientation = exif_data[exif_orientation_tag]

        # Handle EXIF Orientation
        if orientation == 1:
            # Normal image - nothing to do!
            pass
        elif orientation == 2:
            # Mirrored left to right
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 3:
            # Rotated 180 degrees
            img = img.rotate(180)
        elif orientation == 4:
            # Mirrored top to bottom
            img = img.rotate(180).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 5:
            # Mirrored along top-left diagonal
            img = img.rotate(-90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 6:
            # Rotated 90 degrees
            img = img.rotate(-90, expand=True)
        elif orientation == 7:
            # Mirrored along top-right diagonal
            img = img.rotate(90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 8:
            # Rotated 270 degrees
            img = img.rotate(90, expand=True)

    return img


app = Flask(__name__)


@app.route('/', methods=['GET'])
def home():
    return render_template('home.html')


@app.route('/result', methods=['POST'])
def result_page():
    output_img_io = colorizer(request=request, img_name="img")
    jpg_as_text = base64.b64encode(output_img_io.getvalue()).decode()
    return render_template('result.html', photo=jpg_as_text)


@app.route('/check_image', methods=['POST'])
def check_image():
    if request.files.get("image"):

        image = request.files["image"].read()
        image = Image.open(io.BytesIO(image))
        input_array = np.asarray(image)

        return jsonify({
            "image_shape": input_array.shape,
            "is_grayscale_image": len(input_array.shape) == 2,
        })


@app.route('/colorize', methods=['POST'])
def colorize():
    if request.files.get("image"):
        output_img_io = colorizer(request=request, img_name="image")
        return send_file(output_img_io, mimetype='image/png')

    else:
        response = jsonify({
            "message": "An image file is required"
        })
        response.status_code = 401
        return response








In [ ]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
